In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df_purchases2020 = pd.read_csv("../data/raw/purchases_individuals_2020.csv")
df_purchases2019 = pd.read_csv("../data/raw/purchases_individuals_2019.csv")

In [2]:
df_purchases = pd.concat([df_purchases2019, df_purchases2020], ignore_index=True) #concatenating both datasets

df_purchases

,DATAFLOW,LAST UPDATE,freq,indic_is,ind_type,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:ISOC_EC_IBUY(1.0),16/06/24 23:00:00,Annual,Last online purchase: between 3 and 12 months ago,"Females, 16 to 74 years old",Percentage of individuals,Albania,2018,2.14,NaN,NaN
1,ESTAT:ISOC_EC_IBUY(1.0),16/06/24 23:00:00,Annual,Last online purchase: between 3 and 12 months ago,"Females, 16 to 74 years old",Percentage of individuals,Albania,2019,1.82,NaN,NaN
2,ESTAT:ISOC_EC_IBUY(1.0),16/06/24 23:00:00,Annual,Last online purchase: between 3 and 12 months ago,"Females, 16 to 74 years old",Percentage of individuals,Austria,2010,10.89,NaN,NaN
3,ESTAT:ISOC_EC_IBUY(1.0),16/06/24 23:00:00,Annual,Last online purchase: between 3 and 12 months ago,"Females, 16 to 74 years old",Percentage of individuals,Austria,2011,8.58,NaN,NaN
4,ESTAT:ISOC_EC_IBUY(1.0),16/06/24 23:00:00,Annual,Last online purchase: between 3 and 12 months ago,"Females, 16 to 74 years old",Percentage of individuals,Austria,2012,8.74,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
64647,ESTAT:ISOC_EC_IB20(1.0),17/12/24 11:00:00,Annual,Last online purchase: in the last 3 months,"Individuals, 75 years old or more",Percentage of individuals,Norway,2022,35.71,NaN,NaN
64648,ESTAT:ISOC_EC_IB20(1.0),17/12/24 11:00:00,Annual,Last online purchase: in the last 3 months,"Individuals, 75 years old or more",Percentage of individuals,Norway,2023,32.17,NaN,NaN
64649,ESTAT:ISOC_EC_IB20(1.0),17/12/24 11:00:00,Annual,Last online purchase: in the last 3 months,"Individuals, 75 years old or more",Percentage of individuals,Norway,2024,47.58,NaN,NaN
64650,ESTAT:ISOC_EC_IB20(1.0),17/12/24 11:00:00,Annual,Last online purchase: in the last 3 months,"Individuals, 75 years old or more",Percentage of individuals,Sweden,2023,NaN,u,NaN


In [3]:
def empty_cleaner(df):
    df_cleaned = df.dropna(axis=1, how='all') #removing all columns with all NaN values
    df_cleaned = df_cleaned.dropna(axis=1, how='all') #removing all columns with all NaN values
    return df_cleaned
    
df_purchases_cleaned = empty_cleaner(df_purchases) # apply the cleanning function

df_purchases_cleaned

,DATAFLOW,LAST UPDATE,freq,indic_is,ind_type,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:ISOC_EC_IBUY(1.0),16/06/24 23:00:00,Annual,Last online purchase: between 3 and 12 months ago,"Females, 16 to 74 years old",Percentage of individuals,Albania,2018,2.14,NaN
1,ESTAT:ISOC_EC_IBUY(1.0),16/06/24 23:00:00,Annual,Last online purchase: between 3 and 12 months ago,"Females, 16 to 74 years old",Percentage of individuals,Albania,2019,1.82,NaN
2,ESTAT:ISOC_EC_IBUY(1.0),16/06/24 23:00:00,Annual,Last online purchase: between 3 and 12 months ago,"Females, 16 to 74 years old",Percentage of individuals,Austria,2010,10.89,NaN
3,ESTAT:ISOC_EC_IBUY(1.0),16/06/24 23:00:00,Annual,Last online purchase: between 3 and 12 months ago,"Females, 16 to 74 years old",Percentage of individuals,Austria,2011,8.58,NaN
4,ESTAT:ISOC_EC_IBUY(1.0),16/06/24 23:00:00,Annual,Last online purchase: between 3 and 12 months ago,"Females, 16 to 74 years old",Percentage of individuals,Austria,2012,8.74,NaN
...,...,...,...,...,...,...,...,...,...,...
64647,ESTAT:ISOC_EC_IB20(1.0),17/12/24 11:00:00,Annual,Last online purchase: in the last 3 months,"Individuals, 75 years old or more",Percentage of individuals,Norway,2022,35.71,NaN
64648,ESTAT:ISOC_EC_IB20(1.0),17/12/24 11:00:00,Annual,Last online purchase: in the last 3 months,"Individuals, 75 years old or more",Percentage of individuals,Norway,2023,32.17,NaN
64649,ESTAT:ISOC_EC_IB20(1.0),17/12/24 11:00:00,Annual,Last online purchase: in the last 3 months,"Individuals, 75 years old or more",Percentage of individuals,Norway,2024,47.58,NaN
64650,ESTAT:ISOC_EC_IB20(1.0),17/12/24 11:00:00,Annual,Last online purchase: in the last 3 months,"Individuals, 75 years old or more",Percentage of individuals,Sweden,2023,NaN,u


In [4]:
df_purchases_cleaned["OBS_FLAG"].value_counts() # checking values on OBS_FLAG

OBS_FLAG
u     2573
b     1360
e      250
bu      90
Name: count, dtype: int64

In [5]:
df_purchases_cleaned_v2 = df_purchases_cleaned.drop(columns = ["DATAFLOW", "LAST UPDATE", "freq", "unit", "OBS_FLAG"]) # dropping not required columns

df_purchases_cleaned_v2['indic_is'] = df_purchases_cleaned_v2['indic_is'].str.replace("Last online purchase: ", "", regex=False) # removing "Last online purchase: " from column indic_is

df_purchases_cleaned_v2

,indic_is,ind_type,geo,TIME_PERIOD,OBS_VALUE
0,between 3 and 12 months ago,"Females, 16 to 74 years old",Albania,2018,2.14
1,between 3 and 12 months ago,"Females, 16 to 74 years old",Albania,2019,1.82
2,between 3 and 12 months ago,"Females, 16 to 74 years old",Austria,2010,10.89
3,between 3 and 12 months ago,"Females, 16 to 74 years old",Austria,2011,8.58
4,between 3 and 12 months ago,"Females, 16 to 74 years old",Austria,2012,8.74
...,...,...,...,...,...
64647,in the last 3 months,"Individuals, 75 years old or more",Norway,2022,35.71
64648,in the last 3 months,"Individuals, 75 years old or more",Norway,2023,32.17
64649,in the last 3 months,"Individuals, 75 years old or more",Norway,2024,47.58
64650,in the last 3 months,"Individuals, 75 years old or more",Sweden,2023,NaN


In [6]:
df_purchases_cleaned_v3 = df_purchases_cleaned_v2.pivot(index=['indic_is', "ind_type", 'geo'], columns='TIME_PERIOD', values='OBS_VALUE').reset_index() #pivoting to add years on columns

df_purchases_cleaned_v3.columns.name = None # remove the name of the column index for better visualization

df_purchases_cleaned_v3.rename(columns={"geo": "country", "indic_is": "last_online_purchase", "ind_type": "grouped_individuals"}, inplace=True) # better names on columns :)

#also removing the values "Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)" or "European Union - 27 countries (from 2020)"
df_purchases_cleaned_v3 = df_purchases_cleaned_v3[~df_purchases_cleaned_v3['country'].isin(["Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)", "European Union - 27 countries (from 2020)"])]


df_purchases_cleaned_v3

,last_online_purchase,grouped_individuals,country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,between 3 and 12 months ago,All individuals,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.93,1.85,8.81,7.62,16.17,16.65,NaN
1,between 3 and 12 months ago,All individuals,Austria,10.76,9.28,9.67,8.24,9.94,11.35,9.92,8.40,7.63,7.97,10.71,9.01,8.83,9.76,6.66
2,between 3 and 12 months ago,All individuals,Belgium,11.55,12.34,12.18,11.86,13.32,12.97,10.47,11.08,11.50,11.56,8.07,9.05,12.24,10.28,10.76
3,between 3 and 12 months ago,All individuals,Bosnia and Herzegovina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.44,4.65,9.96,9.23,NaN,9.18,9.13
4,between 3 and 12 months ago,All individuals,Bulgaria,1.84,2.10,3.67,4.35,6.55,6.69,5.54,6.46,7.81,7.66,15.33,13.45,16.62,18.26,16.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,more than a year ago or never,"Males, 16 to 74 years old",Spain,44.05,43.18,41.53,43.61,39.82,37.44,38.12,33.97,32.39,32.01,29.70,26.08,26.39,27.02,27.74
5031,more than a year ago or never,"Males, 16 to 74 years old",Sweden,26.48,22.21,19.82,23.98,19.56,21.64,20.88,15.43,14.56,12.54,10.78,8.02,7.07,6.54,7.53
5032,more than a year ago or never,"Males, 16 to 74 years old",Switzerland,NaN,NaN,NaN,NaN,22.79,NaN,NaN,17.95,NaN,15.66,NaN,15.39,NaN,15.65,NaN
5033,more than a year ago or never,"Males, 16 to 74 years old",Türkiye,44.92,NaN,47.07,46.54,46.41,47.57,50.38,51.35,50.82,47.64,48.40,44.09,43.60,42.53,43.20


In [7]:
# in case we want to have the main data source filtered by European regions:

df_per_region = df_purchases_cleaned_v3.copy()

from dictionaries import regions # importing regions dictionary from dictionaries.py

# Create a reverse mapping from country to region
country_to_region = {country: region for region, countries in regions.items() for country in countries}

# Add a 'region' column to the new DataFrame
df_per_region['region'] = df_per_region['country'].map(country_to_region)


df_per_region

,last_online_purchase,grouped_individuals,country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,region
0,between 3 and 12 months ago,All individuals,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.93,1.85,8.81,7.62,16.17,16.65,NaN,Eastern Europe
1,between 3 and 12 months ago,All individuals,Austria,10.76,9.28,9.67,8.24,9.94,11.35,9.92,8.40,7.63,7.97,10.71,9.01,8.83,9.76,6.66,Western Europe
2,between 3 and 12 months ago,All individuals,Belgium,11.55,12.34,12.18,11.86,13.32,12.97,10.47,11.08,11.50,11.56,8.07,9.05,12.24,10.28,10.76,Western Europe
3,between 3 and 12 months ago,All individuals,Bosnia and Herzegovina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.44,4.65,9.96,9.23,NaN,9.18,9.13,Eastern Europe
4,between 3 and 12 months ago,All individuals,Bulgaria,1.84,2.10,3.67,4.35,6.55,6.69,5.54,6.46,7.81,7.66,15.33,13.45,16.62,18.26,16.43,Eastern Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,more than a year ago or never,"Males, 16 to 74 years old",Spain,44.05,43.18,41.53,43.61,39.82,37.44,38.12,33.97,32.39,32.01,29.70,26.08,26.39,27.02,27.74,Southern Europe
5031,more than a year ago or never,"Males, 16 to 74 years old",Sweden,26.48,22.21,19.82,23.98,19.56,21.64,20.88,15.43,14.56,12.54,10.78,8.02,7.07,6.54,7.53,Northern Europe
5032,more than a year ago or never,"Males, 16 to 74 years old",Switzerland,NaN,NaN,NaN,NaN,22.79,NaN,NaN,17.95,NaN,15.66,NaN,15.39,NaN,15.65,NaN,Western Europe
5033,more than a year ago or never,"Males, 16 to 74 years old",Türkiye,44.92,NaN,47.07,46.54,46.41,47.57,50.38,51.35,50.82,47.64,48.40,44.09,43.60,42.53,43.20,Southern Europe


In [ ]:
# create a dummy function to send to the exploration notebook -Esteban <3
def import_df():
    return df_purchases_cleaned_v3, df_per_region